### Prediction with Trained ANN Model

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle

In [7]:
## load all the train model, scaler and encoders
model=load_model('../data/models/ann_model.h5')

# load the encoder and scaler
with open('../data/models/scaler.pkl','rb') as f:
    scaler=pickle.load(f)

with open('../data/models/onehot_encoder_geography.pkl','rb') as f:
    onehot_encoder_geography=pickle.load(f)

with open('../data/models/label_encoder_gender.pkl','rb') as f:
    label_encoder_gender=pickle.load(f)

In [9]:
# Example input data for prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [12]:
## one hot encode the geography column
geography_encoded=onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geography_df=pd.DataFrame(geography_encoded,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))

## label encode the gender column
gender_encoded=label_encoder_gender.transform([input_data['Gender']])

d:\DeepLearning\Customer-Churn-Prediction-using-Artificial-Neural-Networks\.ann\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [13]:
geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:

input_data=pd.DataFrame([input_data]) 


input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,5,50000,2,1,1,60000


In [15]:
# Encode categorical variables
input_data['Gender']=label_encoder_gender.transform([input_data['Gender']])
input_data

d:\DeepLearning\Customer-Churn-Prediction-using-Artificial-Neural-Networks\.ann\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,5,50000,2,1,1,60000


In [16]:
## concat one hot encoded 
input_data=pd.concat([input_data,geography_df],axis=1)
input_data.drop('Geography',axis=1,inplace=True)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,5,50000,2,1,1,60000,1.0,0.0,0.0


In [17]:
## scaling the input data
input_data_scaled=scaler.transform(input_data)
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.00134472, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [18]:
## Prediction Churn
predictions = model.predict(input_data_scaled)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


array([[0.02978558]], dtype=float32)

In [20]:
## prediction_probability
prediction_probability = predictions[0][0]


In [21]:
prediction_probability

np.float32(0.029785577)

In [22]:
if prediction_probability > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_probability:.2f}")
else:
    print(f"The customer is not likely to churn with a probability of {1 - prediction_probability:.2f}")

The customer is not likely to churn with a probability of 0.97
